<a href="https://colab.research.google.com/github/dswh/lil_nlp_with_tensorflow/blob/main/02_02_begin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings for Sentiment Analysis

This notebook explains how to add the embeddings layer to the neural network. We will train our own word embeddings using a simple Keras model for a sentiment classification task.

Steps include:
1. Downloading data from tensorflow dataset.
2. Segregating training and testing sentences & labels.
3. Data preparation to padded sequences
4. Defining out Keras model with an Embedding layer.
5. Train the model and explore the weights from the embedding layer.


In [3]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.13.0


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [5]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 43.52 url/s]


Dataset imdb_reviews downloaded and prepared to /home/codespace/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Segregating training and testing sets

In [10]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [11]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [12]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [14]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type='post'
oov_tok = "<oov>"


In [15]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [16]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 6)                 14406     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 174413 (681.30 KB)
Trainable params: 174413 (681.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Training

In [18]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded,test_labels)
    )

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.4859 - accuracy: 0.7443 - val_loss: 0.3179 - val_accuracy: 0.8634
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2197 - accuracy: 0.9189 - val_loss: 0.3433 - val_accuracy: 0.8525
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0788 - accuracy: 0.9810 - val_loss: 0.4526 - val_accuracy: 0.8359
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0204 - accuracy: 0.9967 - val_loss: 0.4907 - val_accuracy: 0.8414
Epoch 5/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0052 - accuracy: 0.9997 - val_loss: 0.5477 - val_accuracy: 0.8423
Epoch 6/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.5920 - val_accuracy: 0.8435
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 7.9087e-04 - accuracy: 1.0000 - val_loss: 0.6307 - val_accuracy: 0.8441
Ep

## Deriving weights from the embedding layer

In [20]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(10000, 16)
[[-0.01939439 -0.01240165  0.02009435 ...  0.01837049 -0.02044183
  -0.02350586]
 [-0.04679072  0.03699581  0.01880578 ... -0.01742495 -0.03437529
   0.00100436]
 [-0.05009941  0.00095618 -0.01028922 ...  0.03309724  0.02798052
   0.00495312]
 ...
 [-0.11959433 -0.10383242 -0.09520094 ...  0.06797974 -0.03623205
   0.10580233]
 [-0.03147877 -0.03012336 -0.09755279 ...  0.0025449  -0.05407388
  -0.03598964]
 [-0.02918534 -0.07233145 -0.1386606  ...  0.09524831 -0.04338828
   0.096476  ]]
